# Capella API: Tasking Tutorial

In [ ]:
# Required libraries:
# requests
# json
# geojsonio

Your username and password must be saved in a .json file named 'credentials.json' and formatted as follows.

{"username": "yourusername","password": "xxxxxxxxx"}

In [ ]:
!pip install geojsonio # skip this if the library is already installed

### Import required libraries, build a print utility function, assign API endpoints and load Credentials

In [ ]:
import requests
import json

# Helper function to printformatted JSON using the json module
def p(data):
    print(json.dumps(data, indent=2))

# Capella API endpoints
URL = 'https://api.data.capellaspace.com'
token = '/token'
task = '/task/'
user = '/user/'

# Load my username and password
with open('credentials.json') as f:
    data = json.load(f)
    username = data['username']
    password = data['password']

### Get and Print Access Token

In [ ]:
# Get the token
r = requests.post(URL + token, 
                  headers = {'Content-Type': 'application/x-www-form-urlencoded'}, auth=(username,password))

accesstoken = r.json()["accessToken"]

# Print the token
print("Access Token: " + accesstoken)

### Get a User ID

In [ ]:
# Get user ID
headers = {'Authorization':'Bearer ' + accesstoken}
r = requests.get(URL + user, headers=headers)
userid = r.json()["id"]
p(userid)


### Post a Tasking Request

In [ ]:
# Specify tasking request location geometry and properties
taskingrequest = {
    "geometry": {
      "coordinates": [
        -122.40340769290924,
        37.77348366900006,
      ],
      "type": "Point"
    },
    "properties": {
      "userId": userid,
      "collectMode": "stripmap",# 
      "taskingrequestDescription": "Capella office - San Francisco, CA",
      "taskingrequestName": "Capella HQ",
      "windowOpen": "2019-12-13T00:00:00.000Z",# Set this to some future time from now
      "windowDuration": 64800,# time in seconds
      "taskingOption": "standard",
    },
    "type": "Feature"
}
headers = {'Authorization':'Bearer ' + accesstoken}
r = requests.post(URL + task, json=taskingrequest, headers=headers)
requestid = r.json()["taskingrequestId"]

# Print the response
p(r.json())

### Get Submitted Tasking Request Status and Display tasking Location

In [ ]:
# Return the status of the submitted tasking request and map the location
r = requests.get(URL + task + requestid, params={}, headers=headers)
print(r.json())

# Show the tasking location in geojsonio
import geojsonio
url = geojsonio.display(r.text)

### Get All Tasking Requests for User ID and Check Status

In [ ]:
# Return all tasking requests for a customer id and check the status
r = requests.get(URL + task, params={"customerId":userid}, headers=headers)

# Loop over all the features from the response
geojson = r.json()
for f in geojson:
    # Print the ID for each feature
    p(f["properties"]["taskingrequestId"])
    p(f["properties"]["requestStatus"])